# **ANSWERING QUESTION**
---

Nhiệm vụ phần này: Đặt các câu hỏi có ý nghĩa, giải thích tại sao cần trả lời câu hỏi này và phân tích để trả lời câu hỏi.

## CÂU 1: VỚI TỪNG THỂ LOẠI KHÁC NHAU THÌ ĐÁNH GIÁ CỦA KHÁN GIẢ KHÁC NHAU NHƯ THẾ NÀO ? 

Mỗi thể loại phim sẽ thu hút các nhóm khán giả khác nhau. Trả lời câu hỏi này giúp chúng ta tìm hiểu đánh giá của khán giả nói chung với từng thể loại phim khác nhau, từ đó phần nào diễn giáỉ sở thích, khẩu vị xem phim của họ nhằm hỗ trợ trong việc phát triển nội dung phù hợp với thị hiếu. 

Để trả lời câu hỏi này chúng ta sẽ sử dụng dữ liệu của cột **IMDb Ratings**, **IMDb Votes** và **13 cột thể loại phim** để tính đánh giá trung bình và số lượt bình chọn cho từng thể loại (Coi như các thể loại phim có đóng góp như nhau vào sự đánh giá của khán giả của bộ phim đó). 

- Với từng thể loại phim, lọc ra các bộ phim thuộc thể loại đó (cột thể loại có giá trị 'True') để lấy ratings và vote.
- Tính trung bình cho rating và vote của từng bộ phim
- Sắp xép theo thứ tự rating và vote từ cao tới thấp theo thể loại.
- Đưa ra nhận xét/kết luận từ biểu đò.

## CÂU 2: VÀO CÁC THỜI ĐIỂM TRONG NĂM (THÁNG/QUÝ) THÌ THỂ LOẠI PHIM NÀO ĐƯỢC RA MẮT NHIỀU NHẤT ? (giống câu tú gòi =)))

Trả lời câu hỏi này giúp chúng ta tìm hiểu về tương quan giữa thể loại và thời điểm nó được ra mắt, từ đó tìm hiểu về xu hướng phim theo các thời điểm đặc biệt trong năm (theo mùa / theo quý, theo dịp đặc biệt).

Để trả lời câu hỏi này chúng ta sẽ sử dụng dữ liệu của cột **Released time** và 13 cột Thể loại để xác định top 5 thể loại ra mắt nhiều nhất vào từng thời điểm.

## Câu 3: Các phim thuộc thể loại nào có nhiều khả năng được đề cử hoặc giành giải thưởng nhất?

Trả lời câu hỏi này: 
- Giúp các nhà làm phim tập trung sản xuất những nội dung phù hợp để tối đa hóa cơ hội thành công.
- Giúp khán giả lựa chọn các bộ phim theo sở thích và tiềm năng chất lượng.

- Tính tổng số "Win" và "Nomination" theo từng thể loại.
- Tính tỷ lệ giữa số phim giành giải và tổng số phim trong từng thể loại.
- Biểu đồ để minh họa sự phân bố giải thưởng theo các thể loại như "Drama", "Biography/History/War", "Animation"...

## Câu 4: Xếp hạng tuổi (Age rating) ảnh hưởng thế nào đến mức độ phổ biến (số lượt bình chọn IMDb Votes)?

Trả lời câu hỏi này: 
- Hiểu tác động của đối tượng khán giả mục tiêu đến mức độ phổ biến của phim.
- Giúp điều chỉnh nội dung phù hợp với đối tượng mục tiêu để tăng sức hút thị trường.

- Nhóm dữ liệu theo cột Age rating và tính trung bình số lượng IMDb Votes.
- So sánh mức độ phổ biến giữa các nhóm tuổi khác nhau.
- Có thể mở rộng để xem xét tương quan giữa Age rating và các yếu tố khác như IMDb Rating, thể loại.

## CÂU 5: Thời lượng phim (Runtime) ảnh hưởng như thế nào đến số lượng IMDb Votes và IMDb Rating?

Thời lượng phim là một yếu tố khá quan trọng vì nó ảnh hưởng đến trải nghiệm của người xem. Thời lượng dài có thể giúp phim phát triển cốt truyện chi tiết hơn, nhưng cũng tiềm ẩn nhiều nguy cơ như có thể làm giảm hứng thú nếu không được xây dựng tốt và tốn kém nhân lực và nguồn lực vì cần đầu tư nhiều hơn.

Trả lời câu hỏi này giúp chúng ta xác định thời lượng lý tưởng của một bộ phim để thu hút khán giả và đạt được điểm đánh giá cao.


Để trả lời câu hỏi này ta sẽ sử dụng 3 cột là **Runtime, IDMb Votes** và **IMDb Rating**
- Phân tích phân phối thời lượng phim.
- Kiểm tra sự ảnh hưởng của thời lượng đến số lượt bình chọn và đánh giá. Liệu phim có thời lượng thế nào có xu hướng đạt được nhiều bình chọn hoặc đánh giá hơn không? 

## CÂU 6: Thể loại nào có xu hướng phát triển mạnh nhất qua các năm dựa trên số lượng phim phát hành?

Khi theo dõi xu hướng thị trường thì số lượng phim phát hành qua các năm sẽ phản ảnh được sự phát triển của từng thể loại trong ngành điện ảnh. Từ đó, giúp ta dự đoán thể loại nào sẽ tiếp tục phát triển, có khả năng phát triển hay sẽ trở nên lỗi thời. 

Trả lời câu hỏi này giúp dự đoán xu hướng phát triển của ngành công nghiệp điện ảnh để định hướng đầu tư vào các thể loại tiềm năng trong tương lai.

Để trả lời câu hỏi này ta cần 2 cột là **Release time** và 13 cột thể loại
- Tính tổng số lượng phim phát hành mỗi năm theo từng thể loại.
- Trực quan hóa xu hướng của mỗi thể loại qua các năm để so sánh sự thay đổi về số lượng phim phát hành của từng thể loại.
- Phân tích sự tăng trưởng hay suy giảm của từng thể loại để xác định thể loại nào tăng trưởng nhanh hay đang xuống dốc
- Uớc tính số lượng phim phát hành của từng thể loại trong các năm tiếp theo.

## CÂU 7: Mối liên hệ giữa `IMDb Rating` và số phiếu đánh giá `IMDb Votes` là gì?

Trả lời câu hỏi này sẽ giúp ta tìm hiểu mối quan hệ giữa `IMDb Rating` và `IMDb Votes` giúp phân tích liệu mức độ phổ biến (thông qua số lượng phiếu bầu) có ảnh hưởng đến chất lượng đánh giá hay không. Nếu có, điều này có thể giúp dự đoán phim nào sẽ được yêu thích hơn hoặc đạt rating cao hơn dựa trên xu hướng phiếu bầu.

Để trả lời câu hỏi này, cần sử dụng các cột: `IMDb Rating`, `IMDb Votes`

Cách làm:
- Vẽ biểu đồ scatter plot giữa 2 cột để trực quan hóa mối quan hệ
- Tính hệ số tương quan để xác định mức độ ảnh hưởng

## CÂU 8: Thể loại phim nào có xu hướng phù hợp với từng nhóm tuổi `Age rating`?

**Lợi ích của câu hỏi:**

- ***Định hướng sản xuất nội dung:*** Biết được nhóm tuổi nào yêu thích thể loại nào giúp nhà sản xuất phim tập trung phát triển nội dung phù hợp với đối tượng khán giả, tăng khả năng thành công về doanh thu và đánh giá.

- ***Cải thiện chiến lược marketing:*** Các chiến dịch quảng bá phim có thể được tối ưu hóa bằng cách nhấn mạnh thể loại phù hợp với đối tượng mục tiêu, giúp tiết kiệm chi phí và đạt hiệu quả tối đa.

- ***Nắm bắt xu hướng thị trường:*** Phát hiện sự thay đổi trong sở thích phim của từng nhóm tuổi qua thời gian, hỗ trợ các nhà làm phim hoặc nền tảng phân phối trong việc cập nhật danh mục phim.

- ***Hỗ trợ phân loại và kiểm duyệt:*** Các cơ quan kiểm duyệt phim có thể dựa vào thông tin này để đánh giá mức độ phù hợp của một thể loại đối với các nhóm tuổi khác nhau.

**Cách thực hiện:**

- Cột cần thiết: `Age rating` và 13 cột thể loại

- Tính tần suất xuất hiện của từng thể loại

- Vẽ biểu đồ heatmap với các hàng là nhóm tuổi và các cột là thể loại. Màu sắc thể hiện tần suất hoặc tỷ lệ của từng thể loại trong nhóm tuổi.

## CÂU 9: Khoảng thời lượng phim nào phổ biến nhất (ví dụ: 10–20 phút, 20–30 phút, ...) và số lượng phim trong khoảng đó là bao nhiêu?

Trả lời câu hỏi này: 
- Giúp phân tích xu hướng về độ dài của phim, từ đó hiểu rõ hơn về sự lựa chọn phổ biến của các nhà sản xuất và thị hiếu khán giả.
- Hữu ích cho các nhà làm phim khi lên kế hoạch sản xuất nếu họ muốn sản xuất phim phù hợp với xu hướng hiện tại.

## CÂU 10: Tháng nào trong năm được tổng số lượt bình chọn người dùng nhiều nhất?

Trả lời câu hỏi này: 
- Xác định các tháng mà khán giả dành nhiều thời gian hơn để xem phim và bình chọn trên IMDb, từ đó phản ánh mức độ quan tâm của khán giả trong từng tháng
- Giúp nhà sản xuất lên lịch phát hành phim vào các tháng có tương tác cao nhất, nhằm tối ưu hóa lượt xem và đánh giá